In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from zoneinfo import ZoneInfo

In [ ]:
df = pd.read_csv('../data_csv/raw_250101.csv')
df = df.sort_values(by='dt')
df['utc8'] = pd.to_datetime(df['dt'], unit='s', utc=True).dt.tz_convert('Asia/Kuala_Lumpur')
df['year']       = df['utc8'].dt.year
df['month']      = df['utc8'].dt.month
df['day']        = df['utc8'].dt.day
df['hour']       = df['utc8'].dt.hour
df['minute']     = df['utc8'].dt.minute
df['second']     = df['utc8'].dt.second
df['date']       = df['utc8'].dt.date
df['time']       = df['utc8'].dt.time
df['weekday']    = df['utc8'].dt.day_name()     # e.g., 'Monday'
df['week']       = df['utc8'].dt.isocalendar().week
df['day_of_year'] = df['utc8'].dt.dayofyear
df['quarter']    = df['utc8'].dt.quarter

conditions = [
    df['dt'] <= df['sunrise'],
    (df['dt'] > df['sunrise']) & (df['dt'] <= df['sunset'])
]
choices = [0, 1]
df['sunrise_sunset'] = np.select(conditions, choices, default=2)
df['sun_on'] = np.select(conditions, choices, default=0)
df['pressure_norm'] = (df['pressure'] - df['pressure'].mean()) / df['pressure'].std()
df['humidity_stan'] = (df['humidity'] - df['humidity'].min()) / (df['humidity'].max() - df['humidity'].min())
df['dewpoint_stan'] = (df['dew_point'] - df['dew_point'].min()) / (df['dew_point'].max() - df['dew_point'].min())
df

,lat,lon,timezone,timezone_offset,dt,sunrise,sunset,temp,feels_like,pressure,...,minute,second,date,time,weekday,week,day_of_year,quarter,sunrise_sunset,sun_on
254,3.033,101.45,Asia/Kuala_Lumpur,28800,1735660800,1735687143,1735730158,25.78,26.87,1012,...,0,0,2025-01-01,00:00:00,Wednesday,1,1,1,0,0
790,3.033,101.45,Asia/Kuala_Lumpur,28800,1735664400,1735687143,1735730158,25.53,26.59,1012,...,0,0,2025-01-01,01:00:00,Wednesday,1,1,1,0,0
734,3.033,101.45,Asia/Kuala_Lumpur,28800,1735668000,1735687143,1735730158,25.45,26.51,1011,...,0,0,2025-01-01,02:00:00,Wednesday,1,1,1,0,0
617,3.033,101.45,Asia/Kuala_Lumpur,28800,1735671600,1735687143,1735730158,25.15,26.18,1011,...,0,0,2025-01-01,03:00:00,Wednesday,1,1,1,0,0
140,3.033,101.45,Asia/Kuala_Lumpur,28800,1735675200,1735687143,1735730158,24.98,25.99,1010,...,0,0,2025-01-01,04:00:00,Wednesday,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,3.033,101.45,Asia/Kuala_Lumpur,28800,1738839600,1738798094,1738841300,25.08,26.10,1007,...,0,0,2025-02-06,19:00:00,Thursday,6,37,1,1,1
439,3.033,101.45,Asia/Kuala_Lumpur,28800,1738843200,1738798094,1738841300,27.28,29.34,1008,...,0,0,2025-02-06,20:00:00,Thursday,6,37,1,2,0
698,3.033,101.45,Asia/Kuala_Lumpur,28800,1738846800,1738798094,1738841300,26.73,28.41,1008,...,0,0,2025-02-06,21:00:00,Thursday,6,37,1,2,0
881,3.033,101.45,Asia/Kuala_Lumpur,28800,1738850400,1738798094,1738841300,25.08,25.94,1009,...,0,0,2025-02-06,22:00:00,Thursday,6,37,1,2,0
